In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
spark.version

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [ ]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [ ]:
df.count()

In [ ]:
df = df.repartition(4)

In [ ]:
df.write.parquet('/tmp/yellow/2024/10/')

In [ ]:
df.show(10)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df = df \
    .withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.tpep_dropoff_datetime)) 

In [ ]:
df.filter( \
    (df.pickup_date == '2024-10-15') & (df.dropoff_date == '2024-10-15')) \
    .count()

In [ ]:
df.schema

In [ ]:
df = df.withColumn('trip_duration', (F.col('tpep_dropoff_datetime') - F.col('tpep_pickup_datetime')).cast('long'))

In [ ]:
df.select((F.max('trip_duration'))/3600).show()

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [ ]:
df_zone = spark.read.csv('taxi_zone_lookup.csv', header = 'True', inferSchema = 'True')

In [ ]:
df_zone.show()

In [ ]:
df = df.join(df_zone, df.PULocationID == df_zone.LocationID)

In [ ]:
from pyspark.sql.functions import count, desc, asc

In [ ]:
df.groupBy('Zone') \
  .agg(count('*').alias('Count')) \
  .sort(asc('Count')) \
  .show()